In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install wandb -qqq

In [3]:
from wandb.keras import WandbCallback
import wandb

In [4]:
# Configure the sweep
sweep_config = {
    'name' : 'Weight Decay Sweep',
    'method': 'grid',
    'parameters': {
        'weight_decay': {
            'values': [0.0001, 0.001, 0.01]
        },
    }
}

In [5]:
WANDB_API_KEY = '002b3552fd78e3559e38c7317ef0d9eefd0c7c6d'
sweep_id = wandb.sweep(sweep_config, project='nn_project_tuning')

Create sweep with ID: rko63m53
Sweep URL: https://wandb.ai/burntice/nn_project_tuning/sweeps/rko63m53


In [6]:
# sweep reference: 
# https://colab.research.google.com/drive/1gKixa6hNUB8qrn1CfHirOfTEQm0qLCSS?usp=sharing

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import tensorflow as tf


# Load dataset as dataframe
base_path = '/content/drive/My Drive/NN Project/'
df = pd.read_csv(base_path + 'aligned_gender.txt', sep='\t')
df['datadir'] = base_path + df['datadir'].astype(str)

# Train test split
train_val_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_val_df, test_size=0.2)

# function to call for hyperparameter sweep
def train():
    # Fix seed
    seed = 7
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # default hyperparameters
    # Use optimal params from first two runs
    defaults = {
        'epochs': 20,
        'batch_size': 64,
        'learning_rate': 5e-4,
        'weight_decay': 0.001,
        'fc1_num_neurons': 512,
        'fc2_num_neurons': 512,
        'fc3_num_neurons': 512,
        'dropout1_rate': 0,
        'dropout2_rate': 0,
        'dropout3_rate': 0,
        'seed': seed,
        'optimizer': 'adam',
        'hidden_activation': 'relu',
        'output_activation': 'sigmoid',
        'loss_function': 'binary_crossentropy',
        'metrics': ['accuracy'],
    }

    wandb.init(
        config=defaults,
        name='Hyperparameter Tuning',
        project='nn_project',
    )
    config = wandb.config

    # Define model
    mobile_net_v2 = tf.keras.applications.MobileNetV2(
        include_top=False,
        pooling='avg',
        weights='imagenet',
        input_shape=(224,224,3),
    )

    fc1 = tf.keras.layers.Dense(
        config.fc1_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    fc2 = tf.keras.layers.Dense(
        config.fc2_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    fc3 = tf.keras.layers.Dense(
        config.fc3_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    bn1 = tf.keras.layers.BatchNormalization()
    bn2 = tf.keras.layers.BatchNormalization()
    bn3 = tf.keras.layers.BatchNormalization()
    bn4 = tf.keras.layers.BatchNormalization()

    dropout1 = tf.keras.layers.Dropout(rate=config.dropout1_rate)
    dropout2 = tf.keras.layers.Dropout(rate=config.dropout2_rate)
    dropout3 = tf.keras.layers.Dropout(rate=config.dropout3_rate)

    model = tf.keras.models.Sequential([
        mobile_net_v2,
        tf.keras.layers.Flatten(),
        bn1,
        fc1,
        dropout1,
        bn2,
        fc2,
        dropout2,
        bn3,
        fc3,
        dropout3,
        bn4,
        tf.keras.layers.Dense(1, activation=config.output_activation),
    ])

    # Compile model 
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
        loss=config.loss_function,
        metrics=config.metrics,
    )

    # Load images into keras image generator 
    datagen_train = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    datagen_val = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    datagen_test = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )

    train_generator = datagen_train.flow_from_dataframe(
        dataframe=train_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    val_generator = datagen_train.flow_from_dataframe(
        dataframe=val_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    test_generator = datagen_test.flow_from_dataframe(
        dataframe=test_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=config.epochs,
        callbacks=[WandbCallback()],
    )

In [8]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: z10aqmof with config:
wandb: 	weight_decay: 0.0001
wandb: Currently logged in as: burntice (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


9412608/9406464 [==============================] - 0s 0us/step
Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
122/122 [==============================] - 3326s 27s/step - loss: 0.5506 - accuracy: 0.8383 - val_loss: 0.5175 - val_accuracy: 0.8785
Epoch 2/20
122/122 [==============================] - 137s 1s/step - loss: 0.3344 - accuracy: 0.9398 - val_loss: 1.5184 - val_accuracy: 0.7658
Epoch 3/20
122/122 [==============================] - 134s 1s/step - loss: 0.2656 - accuracy: 0.9631 - val_loss: 1.8106 - val_accuracy: 0.7350
Epoch 4/20
122/122 [==============================] - 134s 1s/step - loss: 0.2412 - accuracy: 0.9742 - val_loss: 5.0808 - val_accuracy: 0.5971
Epoch 5/20
122/122 [==============================] - 134s 1s/step - loss: 0.2207 - accuracy: 0.9813 - val_loss: 1.3979 - val_accuracy: 0.8073
Epoch 6/20
122/122 [==============================] - 134s 1s/step - loss: 0.2216 - accuracy: 0.9809 - val

epoch,19
loss,0.1084
accuracy,0.99526
val_loss,0.76303
val_accuracy,0.89339
_step,19
_runtime,6025
_timestamp,1606028061
best_val_loss,0.32828
best_epoch,17


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▅▃▃▃▃▂▃▂▂▂▂▂▂▁▂▁▁▁▁
accuracy,▁▆▇▇▇▇█▇████████████
val_loss,▁▃▃█▃▃▅▂▂▃▁▁▁▂▂▁▁▁▂▂
val_accuracy,▇▄▄▁▅▄▄▆▆▅▇▇▇▆▅███▇▇
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██


wandb: Agent Starting Run: llahvttq with config:
wandb: 	weight_decay: 0.001
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

wandb: Agent Starting Run: 5h7exiw8 with config:
wandb: 	weight_decay: 0.01
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
122/122 [==============================] - 136s 1s/step - loss: 13.3249 - accuracy: 0.8362 - val_loss: 9.0761 - val_accuracy: 0.7991
Epoch 2/20
122/122 [==============================] - 134s 1s/step - loss: 5.9036 - accuracy: 0.9346 - val_loss: 3.7600 - val_accuracy: 0.8816
Epoch 3/20
122/122 [==============================] - 134s 1s/step - loss: 2.2497 - accuracy: 0.9560 - val_loss: 2.0762 - val_accuracy: 0.7427
Epoch 4/20
122/122 [==============================] - 133s 1s/step - loss: 0.7728 - accuracy: 0.9757 - val_loss: 0.8592 - val_accuracy: 0.8570
Epoch 5/20
122/122 [==============================] - 133s 1s/step - loss: 0.2983 - accuracy: 0.9760 - val_loss: 1.7670 - val_accuracy: 0.7529
Epoch 6/20
122/122 [==============================] - 133s 1s/step - loss: 0.1689 - accuracy: 0.9730 - val_loss: 4.7824 - val_accuracy: 0.6515
Epoch 7/20
122/122 [=======

epoch,19
loss,0.08744
accuracy,0.98129
val_loss,0.43998
val_accuracy,0.90876
_step,19
_runtime,2768
_timestamp,1606033536
best_val_loss,0.43998
best_epoch,19


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▆▇██▇███████████▇██
val_loss,█▄▂▁▂▅▁▂▂▁▁▂▂▁▂▁▂▁▁▁
val_accuracy,▅▇▄▇▅▂█▃▁▇▆▆▂▇▅▇▁▆▇█
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇██


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
